In [108]:
import requests
import pandas as pd
from tqdm import tqdm

# Сбор данных с сайта hh.ru

In [109]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) ' \
             'Chrome/104.0.5112.124 YaBrowser/22.9.4.863 Yowser/2.5 Safari/537.36'

JOB_TITLE = 'python backend разработчик'
URL = 'https://api.hh.ru/vacancies'

> ### Собираем id вакансий Python Backend dev

In [110]:
def number_of_pages(url: str, job_title: str, per_page=100) -> int:
    '''
    Получаем количество страниц с вакансиями
    '''
    param = {'text': job_title, 'per_page': per_page}
    req = requests.get(url, params=param)
    return req.json()['pages']

In [111]:
def get_job_id(url: str, job_title: str, per_page=100) -> list:
    '''
    Функция возвращает список id вакансий с названием job_title
    '''
    job_id = []
    pages = number_of_pages(url, job_title, per_page)
    for page in tqdm(range(pages)):
        param = {'text': job_title, 'per_page': per_page, 'page': page}
        req = requests.get(url, params=param)
        vacancies = req.json()
        for vac in vacancies.get('items'):
            job_id.append(vac.get('id'))
    return job_id

In [112]:
python_dev_job_id = get_job_id(URL, JOB_TITLE)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.91it/s]


In [113]:
print('Количество найденных вакансий:', len(python_dev_job_id))

Количество найденных вакансий: 910


> ### Собираем подробные описания вакансий в список по найденным id.

In [114]:
list_vacs = []
for id in tqdm(python_dev_job_id):
    url_2 = f'https://api.hh.ru/vacancies/{id}'
    r_2 = requests.get(url_2)
    one_vac = r_2.json()
    list_vacs.append(one_vac)

100%|████████████████████████████████████████████████████████████████████████████████| 910/910 [02:47<00:00,  5.43it/s]


In [115]:
df_vacancies = pd.DataFrame(list_vacs)
df_vacancies.head()

,id,premium,billing_type,relations,name,insider_interview,response_letter_required,area,salary,type,...,suitable_resumes_url,apply_alternate_url,has_test,test,alternate_url,working_days,working_time_intervals,working_time_modes,accept_temporary,languages
0,71240928,False,"{'id': 'standard_plus', 'name': 'Стандарт плюс'}",[],Junior Backend разработчик (Python),None,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'from': None, 'to': 80000, 'currency': 'RUR',...","{'id': 'open', 'name': 'Открытая'}",...,None,https://hh.ru/applicant/vacancy_response?vacan...,False,None,https://hh.ru/vacancy/71240928,[],[],[],False,[]
1,72047249,True,"{'id': 'premium', 'name': 'Премиум'}",[],Backend разработчик Python (Middle/Senior),None,False,"{'id': '2759', 'name': 'Ташкент', 'url': 'http...","{'from': 1500, 'to': None, 'currency': 'USD', ...","{'id': 'open', 'name': 'Открытая'}",...,None,https://hh.ru/applicant/vacancy_response?vacan...,False,None,https://hh.ru/vacancy/72047249,[],[],[],False,"[{'id': 'rus', 'name': 'Русский', 'level': {'i..."
2,71675473,False,"{'id': 'standard_plus', 'name': 'Стандарт плюс'}",[],Разработчик Python,None,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'from': 120000, 'to': 200000, 'currency': 'RU...","{'id': 'open', 'name': 'Открытая'}",...,None,https://hh.ru/applicant/vacancy_response?vacan...,False,None,https://hh.ru/vacancy/71675473,[],[],[],False,[]
3,72034802,False,"{'id': 'standard_plus', 'name': 'Стандарт плюс'}",[],Backend-разработчик jun+/mid- (Golang),None,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'from': 100000, 'to': 400000, 'currency': 'RU...","{'id': 'open', 'name': 'Открытая'}",...,None,https://hh.ru/applicant/vacancy_response?vacan...,False,None,https://hh.ru/vacancy/72034802,[],[],[],True,[]
4,71933927,False,"{'id': 'standard_plus', 'name': 'Стандарт плюс'}",[],Software Developer Python,None,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'from': 60000, 'to': 200000, 'currency': 'RUR...","{'id': 'open', 'name': 'Открытая'}",...,None,https://hh.ru/applicant/vacancy_response?vacan...,False,None,https://hh.ru/vacancy/71933927,[],"[{'id': 'from_four_to_six_hours_in_a_day', 'na...","[{'id': 'start_after_sixteen', 'name': 'Можно ...",True,[]


> ### Оставляем нужные столбцы

In [116]:
df_vacancies.columns

Index(['id', 'premium', 'billing_type', 'relations', 'name',
       'insider_interview', 'response_letter_required', 'area', 'salary',
       'type', 'address', 'allow_messages', 'experience', 'schedule',
       'employment', 'department', 'contacts', 'description',
       'branded_description', 'vacancy_constructor_template', 'key_skills',
       'accept_handicapped', 'accept_kids', 'archived', 'response_url',
       'specializations', 'professional_roles', 'code', 'hidden',
       'quick_responses_allowed', 'driver_license_types',
       'accept_incomplete_resumes', 'employer', 'published_at', 'created_at',
       'initial_created_at', 'negotiations_url', 'suitable_resumes_url',
       'apply_alternate_url', 'has_test', 'test', 'alternate_url',
       'working_days', 'working_time_intervals', 'working_time_modes',
       'accept_temporary', 'languages'],
      dtype='object')

In [117]:
col_filter = ['id', 'name', 'area', 'salary', 'experience', 'schedule', 'employment', 'key_skills', 'employer', 'languages']

In [118]:
df = df_vacancies.copy()

In [119]:
df_vacancies = df_vacancies[col_filter]

> ### Получаем значения из словарей

In [120]:
df_vacancies['area'] = df_vacancies.area.apply(lambda x: x.get('name'))
df_vacancies['experience'] = df_vacancies.experience.apply(lambda x: x.get('name'))
df_vacancies['schedule'] = df_vacancies.schedule.apply(lambda x: x.get('name'))
df_vacancies['employment'] = df_vacancies.employment.apply(lambda x: x.get('name'))
df_vacancies['key_skills'] = df_vacancies.key_skills.apply(lambda x: [i.get('name') for i in x])

In [121]:
df_vacancies['salary_from'] = df_vacancies.salary.apply(lambda x: x.get('from') if x is not None else None)
df_vacancies['salary_to'] = df_vacancies.salary.apply(lambda x: x.get('to') if x is not None else None)
df_vacancies['salary_currency'] = df_vacancies.salary.apply(lambda x: x.get('currency') if x is not None else None)

df_vacancies.drop(['salary'], axis=1, inplace=True)

In [122]:
df_vacancies['employer_id'] = df_vacancies.employer.apply(lambda x: x.get('id'))
df_vacancies['employer_name'] = df_vacancies.employer.apply(lambda x: x.get('name'))

df_vacancies.drop(['employer'], axis=1, inplace=True)

In [123]:
def get_lang(lang: list) -> list:
    return [[i.get('name'), i.get('level').get('name')] for i in lang]

df_vacancies['languages'] = df_vacancies.languages.apply(get_lang)

In [124]:
df_vacancies.head()

,id,name,area,experience,schedule,employment,key_skills,languages,salary_from,salary_to,salary_currency,employer_id,employer_name
0,71240928,Junior Backend разработчик (Python),Москва,От 1 года до 3 лет,Полный день,Полная занятость,"[Python, Linux, PostgreSQL, Django Framework, ...",[],NaN,80000.0,RUR,863273,MillionAgents
1,72047249,Backend разработчик Python (Middle/Senior),Ташкент,От 1 года до 3 лет,Полный день,Полная занятость,"[PostgreSQL, Git, Python, Django Framework, Li...","[[Русский, C1 — Продвинутый], [Английский, B2 ...",1500.0,NaN,USD,2836478,INSANE GROUP CO
2,71675473,Разработчик Python,Москва,От 3 до 6 лет,Удаленная работа,Полная занятость,"[Python, Django Framework, Pandas]",[],120000.0,200000.0,RUR,2544353,Инспектор Клауд
3,72034802,Backend-разработчик jun+/mid- (Golang),Москва,От 1 года до 3 лет,Полный день,Полная занятость,"[Golang, Linux, PostgreSQL, Docker, Python]",[],100000.0,400000.0,RUR,9244855,Центр Развития Технологий
4,71933927,Software Developer Python,Москва,От 1 года до 3 лет,Удаленная работа,Полная занятость,"[Python, PostgreSQL, Django Framework, Linux, ...",[],60000.0,200000.0,RUR,4643769,ВЕБ Инфраструктура
